In [23]:
import pandas as pd 


In [2]:
df = pd.read_csv('커피빈매장.csv')
df

,Name,Address,Tel
0,삼성봉은사거리점,"서울시 강남구 영동대로 607 1,2층",02-3443-5618
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103
2,삼성오크우드점,서울 강남구 테헤란로87길 46 지하 2층,02-3466-8507
3,삼성봉은사로점,서울시 강남구 봉은사로 628 엘슨빌딩1층,02-538-4523
4,도심공항타워점,서울시 강남구 테헤란로87길 36 공항타워 1층,02-2016-5330
...,...,...,...
208,강동경희대병원앞점,서울특별시 강동구 동남로 885 양지빌딩 1층,02-481-9938
209,새문안로점,"서울특별시 종로구 새문안로 91 고려빌딩 1,2층",02-739-5806
210,광화문역1번출구점,서울시 종로구 새문안로5가길 7 1층 일부,02-723-3360
211,성균관대법학관점,서울시 종로구 성균관로 25-2 성균관대학교 법학관 B2,02-740-1958


In [3]:
import requests
import numpy as np

def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    headers = {'Authorization':'KakaoAK 94b799d02f871182dd937e261cfc2783'}
    res = requests.get(url, headers=headers).json()
    if res['documents']:
        x =res['documents'][0]['x']
        y =res['documents'][0]['y']
    else:
        x = np.nan
        y = np.nan
    return x, y
    

In [4]:
(x, y)= get_location('서울시')
(x, y)

('126.978652258309', '37.5668260046608')

In [5]:
store = pd.read_csv('커피빈매장.csv')
store['X'] = ''
store['Y'] = ''
store.head(2)

,Name,Address,Tel,X,Y
0,삼성봉은사거리점,"서울시 강남구 영동대로 607 1,2층",02-3443-5618,,
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103,,


In [6]:
for i in store.index :
    address = store.loc[i, 'Address']
    x, y = get_location(address)
    store.loc[i, 'X'] = x
    store.loc[i, 'Y'] = y

In [7]:
store.isnull().sum()
store.dropna(inplace=True)

In [9]:
store.isnull().sum()

Name       0
Address    0
Tel        0
X          0
Y          0
dtype: int64

In [30]:
store.head()

,Name,Address,Tel,X,Y
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103,127.057115600619,37.5129534255276
2,삼성오크우드점,서울 강남구 테헤란로87길 46 지하 2층,02-3466-8507,127.058205573267,37.5109823973192
3,삼성봉은사로점,서울시 강남구 봉은사로 628 엘슨빌딩1층,02-538-4523,127.064164448221,37.5146694627537
4,도심공항타워점,서울시 강남구 테헤란로87길 36 공항타워 1층,02-2016-5330,127.058597240146,37.5102336492314
5,트레이드타워점,서울시 강남구 영동대로 511 트레이드타워 지하1층,02-803-9994,127.061023782931,37.510356849269


In [31]:
store.to_csv('커피빈맵.csv', index = False)

In [39]:
store = pd.read_csv('커피빈맵.csv')

In [34]:
address = '인천'
filt = store['Address'].str.contains(address)
df = store[filt]
df

,Name,Address,Tel,X,Y
128,송도트리플스트리트점,"인천 연수구 송도과학로 16번길 33-4 D동 119호,207호",032-310-9644,126.662842,37.378661
129,송도홈플러스점,인천광역시 연수구 송도국제대로 165(송도동) 홈플러스송도점1층,032-822-5905,126.656372,37.380226
150,영종도운서역점,"인천시 중구 신도시남로 142번길 6 (메가스타 영종 139,140,141호)",032-746-9822,126.491490,37.493421
151,인천공항 T2랜드3F동편점,인천 중구 제2터미널대로 446 인천공항 제2여객터미널 랜드사이드 3층 동편 (GA...,032-743-2894,126.433209,37.469249
152,인천공항탑승동105gate앞점,인천광역시 중구 공항로272 인천공항 제1여객터미널 탑승동 3층 105GATE앞,032-743-7223,126.427699,37.458351


In [38]:
import folium
import warnings
warnings.filterwarnings(action = 'ignore')

def get_map(address):
    filt = store['Address'].str.contains(address)
    df = store[filt]
    if len(df.index) == 0:
        print('못찾음')
    else : 
        first = df.head(1)
        x = first['X']
        y = first['Y']
        map = folium.Map(location=[y,x], zoom_start=15)
        for idx, row in df.iterrows():
            name= row['Name']
            x = row['X']
            y = row['Y']
            folium.Marker(location=[y,x], 
                          popup=folium.Popup(f'[{idx}]{name}', max_width=200), 
                          icon=folium.Icon(color='red', icon='star')).add_to(map)
        map.save(f'{address}.html')

get_map('인천')